# Day 2
## Part 1
Rather than brute force it, work out the first and last valid "halves" from the given range, bearing in mind numbers with odd numbers of digits will always be valid.

In [1]:
def parse_data(s):
    return [tuple(map(int, x.split("-"))) for x in s.strip().split(",")]

test_data = parse_data("""11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124""".replace("\n", ""))
test_data

[(11, 22),
 (95, 115),
 (998, 1012),
 (1188511880, 1188511890),
 (222220, 222224),
 (1698522, 1698528),
 (446443, 446449),
 (38593856, 38593862),
 (565653, 565659),
 (824824821, 824824827),
 (2121212118, 2121212124)]

In [2]:
import math

def int_length(x):
    if x == 0:
        return 1
    return int(math.log10(x)) + 1

def half_start(x):
    length = int_length(x)
    if length % 2 == 1:
        return 10 ** (length // 2)
    else:
        return x // (10 ** (length // 2))

def half_end(x):
    length = int_length(x)
    if length % 2 == 1:
        return 10 ** (length // 2) - 1
    else:
        return x // (10 ** (length // 2))
        
assert half_start(89) == 8
assert half_start(8957) == 89
assert half_start(895) == 10

assert half_end(89) == 8
assert half_end(8957) == 89
assert half_end(895) == 9
assert half_end(89572) == 99

In [3]:
def repeat_int(x):
    return x * 10 ** int_length(x) + x

assert repeat_int(58) == 5858
assert repeat_int(5) == 55

In [4]:
def invalid_ids(start, end):
    for x in range(half_start(start), half_end(end) + 1):
        y = repeat_int(x)
        if start <= y <= end:
            yield y

list(invalid_ids(5000, 6000))

[5050, 5151, 5252, 5353, 5454, 5555, 5656, 5757, 5858, 5959]

In [5]:
def part_1(data):
    return sum(sum(invalid_ids(a, b)) for a, b in data)

assert part_1(test_data) == 1227775554

In [6]:
data = parse_data(open("input").read())

In [7]:
part_1(data)

12850231731

## Part 2
Generalising the approach above is feasible but fiddly. Let's have a look at the size of the problem.

What's the biggest range?

In [8]:
max(b-a for a, b in data)

236829

What's the longest number?

In [9]:
int_length(max(b for a, b in data))

10

The maximum possible repeated length is then 5, of which there are 99,000. That's not too many, so create a set of every possible invalid number and then intersect with each range.

In [10]:
def repeat_int(x, n=2):
    return sum(x * 10 ** (int_length(x) * i) for i in range(n))

assert repeat_int(1234, 3) == 123412341234

In [11]:
from functools import cache

@cache
def invalid_numbers(max_n=10):
    result = set()
    for repeat_length in range(1, 6):
        for repeat_number in range(2, max_n // repeat_length + 1):
            for x in range(10 ** (repeat_length - 1), 10 ** repeat_length):
                result.add(repeat_int(x, repeat_number))
    return result

In [12]:
def part_2(data):
    return sum(
        sum(set(range(a, b + 1)) & invalid_numbers())
        for a, b in data
    )

assert part_2(test_data) == 4174379265

In [13]:
part_2(data)

24774350322